In [2]:
from typing import *
from analysis.core.currency import Currency
from analysis.core.utils import load_data_from_currencies, compute_log_returns
from analysis.core.time_utils import Bounds
from datetime import date, timedelta


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


# Current portfolio that we have
existing_portfolio: Dict[Currency, float] = {
    Currency.BTC: 0.6086,
    Currency.ETH: 0.105,
    Currency.XRP: 0.049,
    Currency.SOL: 0.0278,
    Currency.TRX: 0.0066,
    Currency.TON: 0.003,
    Currency.LINK: 0.0535,
    Currency.AAVE: 0.0175,
    Currency.ONDO: 0.0171,
    Currency.ENA: 0.0061,
    Currency.MOVE: 0.0057,
    Currency.HYPE: 0.0232,
    Currency.UNI: 0.0162,
    Currency.TAO: 0.0082,
    Currency.MKR: 0.0024,
    Currency.USDT: 0.0167 * 3
}

ModuleNotFoundError: No module named 'analysis'

<h4>Check portfolios of different BTC/ETH combinations similar to VanEck report</h4>

In [ ]:
start_date: date = date(2013, 1, 1)
end_date: date = date(2025, 2, 28)
bounds: Bounds = Bounds.for_days(start_date, end_date)
selected_currencies: List[Currency] = [Currency.BTC, Currency.ETH]

df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=selected_currencies)
df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)

plt.title("Daily Returns BTC vs ETH")
sns.histplot(df_returns["BTC"], label="BTC", stat="probability")
sns.histplot(df_returns["ETH"], label="ETH", stat="probability")

plt.legend()
plt.show()

<h4>BTC and ETH 60%, 40% allocations</h4>

In [ ]:
def compute_return_series_for_portfolio(df_returns: pd.DataFrame, portfolio: Dict[Currency, float]) -> pd.Series:
    currencies: List[str] = [currency.name for currency in portfolio]
    weights: np.ndarray = np.array(portfolio.values())

    return (df_returns[currencies] * weights).sum(axis=1)


portfolio_6040: Dict[Currency, float] = {
    Currency.BTC: 0.6, Currency.ETH: 0.4
}

returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=portfolio_6040)

plt.plot((1 + returns).cumprod())
plt.show()

<h4>Other allocations of BTC/ETH</h4>

In [ ]:
shares: np.ndarray = np.arange(0, 1.01, 0.01)
final_pnl: List[float] = []

for btc_share in np.arange(0, 1.01, 0.01):
    returns: pd.Series = compute_return_series_for_portfolio(
        df_returns=df_returns, 
        portfolio={Currency.BTC: btc_share, Currency.ETH: 1-btc_share}
    )
    final_pnl.append((1 + returns).prod())
    
    plt.plot((1 + returns).cumprod(), color="red", alpha=.1)

plt.title("How BTC/ETH share affects portfolio returns")
plt.show()

In [ ]:
final_pnl = np.array(final_pnl)
optimal_btc_share: float = shares[final_pnl.argmax()]

plt.title("Optimal BTC allocation against ETH")
plt.plot(shares, final_pnl)
plt.axvline(optimal_btc_share, color="red", linestyle="--", label=f"Optimal BTC allocation: {optimal_btc_share}")
plt.legend()
plt.show()

<h4>If we take volatility of returns into account. Max Sharpe portfolio from BTC & ETH</h4>

In [ ]:
# compute max sharpe portfolio
from analysis.portfolio_makers.max_sharpe import MaxSharpePortfolio


optimizer: MaxSharpePortfolio = MaxSharpePortfolio()
optimal_btc_eth_portfolio: Dict[Currency, float] = optimizer.find_portfolio(
    df_returns=df_returns, selected_currencies=[Currency.BTC, Currency.ETH]
)

optimal_btc_eth_portfolio

Получаем похожую картинку, что кстати совпадает с нашим существующим портфелем

<h4>Compute historical stats of existing portfolio</h4>

Так как данных нет для всех активов, мы будем держать кэш до момента, когда появятся торги актива

In [ ]:
today: date = date.today()
start_date: date = today - timedelta(days=180)

bounds: Bounds = Bounds.for_days(start_date, today)
df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=list(existing_portfolio.keys()))
df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
df_returns[Currency.USDT.name] = 0

# fill missing returns with zeroes
df_returns = df_returns.fillna(0)

In [ ]:
portfolio_returns: pd.Series = compute_return_series_for_portfolio(
    df_returns=df_returns, portfolio=existing_portfolio
)
optimal_btceth_returns: pd.Series = compute_return_series_for_portfolio(
    df_returns=df_returns, portfolio=optimal_btc_eth_portfolio
)

plt.title("Существующий портфель против \n оптимального BTC&ETH портфеля")
plt.plot((portfolio_returns + 1).cumprod(), label="Our portfolio")
plt.plot((optimal_btceth_returns + 1).cumprod(), label="Optimal BTC/ETH portfolio")
plt.legend()
plt.xticks(rotation=70)
plt.show()

In [ ]:
from tqdm import tqdm


def compute_multiple_returns(offsets: List[timedelta]) -> pd.DataFrame:
    today: date = date.today()

    returns: List[Dict[str, float]] = []

    for offset in tqdm(offsets):
        bounds: Bounds = Bounds.for_days(today - offset, today)
        df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=list(existing_portfolio.keys()))
        df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
        df_returns[Currency.USDT.name] = 0
        # fill missing returns with zeroes
        df_returns = df_returns.fillna(0)

        # Compute returns of BTC/ETH portfolio and existing portfolio
        existing_portfolio_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=existing_portfolio)
        optimal_btceth_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=optimal_btc_eth_portfolio)

        optimal_btceth_return: float = (optimal_btceth_returns + 1).prod()
        existing_portfolio_return: float = (existing_portfolio_returns + 1).prod()

        returns.append({
            "past": offset.days,
            "btceth_portfolio": optimal_btceth_return,
            "btceth_portfolio_annualized_return": (1 + optimal_btceth_returns.mean()) ** 365 - 1,
            "btceth_portfolio_annualized_vol": optimal_btceth_returns.std() * np.sqrt(365),
            "existing_portfolio": existing_portfolio_return,
            "existing_portfolio_annualized_return": (1 + existing_portfolio_returns.mean())**365 - 1,
            "existing_portfolio_annualized_vol": existing_portfolio_returns.std() * np.sqrt(365),
        })

    return pd.DataFrame(returns)

In [ ]:
offsets: List[timedelta] = [
    timedelta(days=30),
    timedelta(days=90),
    timedelta(days=180),
    timedelta(days=365),
    timedelta(days=365*3),
    timedelta(days=365*5)
]

df_stats = compute_multiple_returns(offsets=offsets)

In [ ]:
df_stats["btceth_sharpe_ratio"] = df_stats["btceth_portfolio_annualized_return"] / df_stats["btceth_portfolio_annualized_vol"]
df_stats["existing_portfolio_sharpe_ratio"] = df_stats["existing_portfolio_annualized_return"] / df_stats["existing_portfolio_annualized_vol"]

<h4>Overall return</h4>

In [ ]:
df_stats[["past", "btceth_portfolio", "existing_portfolio"]]

<h4>Annualized Sharpe ratios</h4>

In [ ]:
df_stats[["past", "btceth_sharpe_ratio", "existing_portfolio_sharpe_ratio"]]

<h4>Annualized returns</h4>

In [ ]:
df_stats[["past", "btceth_portfolio_annualized_return", "existing_portfolio_annualized_return"]]

<h4>Annualized volatility</h4>

In [ ]:
df_stats[["past", "btceth_portfolio_annualized_vol", "existing_portfolio_annualized_vol"]]

<h4>Если мы делаем ребаланс, то есть строим совершенно новые портфели каждый месяц, используя данные за последние 3 месяца</h4>

In [ ]:
def compute_return_series_of_actively_rebalanced_portfolio(
    bounds: Bounds, interval: timedelta, step: timedelta, use_currencies: List[Currency]
) -> pd.Series:
    df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=use_currencies)
    df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
    
    df_returns[Currency.USDT.name] = 0
    # fill missing returns with zeroes
    df_returns = df_returns.fillna(0)
    
    overlapping_bounds: List[Bounds] = bounds.generate_overlapping_bounds(step=step, interval=interval)
    
    returns_tss: List[pd.Series] = []
    
    for sub_bounds in tqdm(overlapping_bounds):
        portfolio: Dict[Currency, float] = optimizer.find_portfolio(
            df_returns=df_returns.loc[sub_bounds.day0: sub_bounds.day1], selected_currencies=list(Currency)
        )
        returns = compute_return_series_for_portfolio(
            df_returns=df_returns.loc[sub_bounds.day1: sub_bounds.day1 + timedelta(days=30)],
            portfolio=portfolio
        )
        returns_tss.append(returns)

    return pd.concat(returns_tss)

In [ ]:
bounds: Bounds = Bounds.for_days(
    start_date=date.today() - timedelta(days=270), end_date=date.today()
)

actively_rebalanced_portfolio_returns: pd.Series = compute_return_series_of_actively_rebalanced_portfolio(
    bounds=bounds, interval=timedelta(days=90), step=timedelta(days=30), use_currencies=list(Currency)
)

In [ ]:
plt.figure(figsize=(16, 10))

plt.title("Сравнение разных портфелей за последний год по доходности")
plt.plot((1 + actively_rebalanced_portfolio_returns).cumprod(), label="Портфель с ежемесячной ребалансировкой")
plt.plot((portfolio_returns + 1).cumprod(), label="Наш нынешний портфель")
plt.plot((optimal_btceth_returns + 1).cumprod(), label="Оптимальный портфель из BTC и ETH")

plt.legend()
plt.xticks(rotation=70)
plt.savefig("three_strategies_compared.png")
plt.show()

In [ ]:
stats: List[Dict[str, Any]] = []

for offset in offsets:
    bounds: Bounds = Bounds.for_days(
        start_date=date.today() - offset, end_date=date.today()
    )
    actively_rebalanced_portfolio_returns: pd.Series = compute_return_series_of_actively_rebalanced_portfolio(
        bounds=bounds, interval=timedelta(days=90), step=timedelta(days=30), use_currencies=list(Currency)
    )
    stats.append({
        "past": offset.days,
        "portfolio_rebalanced_90/30": (actively_rebalanced_portfolio_returns + 1).prod(),
        "portfolio_rebalanced_90/30_annualized_return": (1 + actively_rebalanced_portfolio_returns.mean()) ** 252 - 1,
        "portfolio_rebalanced_90/30_annualized_std": actively_rebalanced_portfolio_returns.std() * np.sqrt(252)
    })

In [ ]:
df_rebalanced = pd.DataFrame(stats)
df_rebalanced["sharpe_ratio"] = (
    df_rebalanced["portfolio_rebalanced_90/30_annualized_return"] / df_rebalanced["portfolio_rebalanced_90/30_annualized_std"]
)

df_rebalanced

<h4>All Sharpe ratios</h4>

In [ ]:
df_all = pd.concat([df_stats, df_rebalanced], axis=1)
df_all[["past", "btceth_sharpe_ratio", "existing_portfolio_sharpe_ratio", "sharpe_ratio"]].iloc[:, 1:]

<h4>All Annualized returns</h4>

In [ ]:
df_all[
    ["past", "btceth_portfolio_annualized_return", "existing_portfolio_annualized_return", "portfolio_rebalanced_90/30_annualized_return"]
].iloc[:, 1:]

<h4>All total returns</h4>

In [ ]:
df_all[
    ["past", "btceth_portfolio", "existing_portfolio", "portfolio_rebalanced_90/30"]
].iloc[:, 1:]

<h4>Get crypto market cap</h4>

In [ ]:
import requests


resp = requests.get(
    "https://api.coinmarketcap.com/data-api/v4/global-metrics/quotes/historical?range=ALL&convertId=2781&module=marketcap"
)

In [ ]:
df_mcap: pd.DataFrame = pd.DataFrame(resp.json()["data"]["points"])
df_mcap["timestamp"] = pd.to_datetime(df_mcap["timestamp"], unit="s")
df_mcap.plot(x="timestamp", y="marketCap")

plt.xticks(rotation=70)
plt.show()

In [ ]:
df_mcap["market_return"] = (df_mcap["marketCap"].shift(-1) - df_mcap["marketCap"]) / df_mcap["marketCap"]

sns.histplot(
    df_mcap["market_return"], stat="probability"
)

In [ ]:
beta_values: List[Dict[str, float]] = []

for offset in tqdm(offsets):
    bounds: Bounds = Bounds.for_days(start_date=date.today() - offset, end_date=date.today())
    
    df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=list(existing_portfolio.keys()))
    df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
    df_returns[Currency.USDT.name] = 0
    # fill missing returns with zeroes
    df_returns = df_returns.fillna(0)

    # Compute returns of BTC/ETH portfolio and existing portfolio
    existing_portfolio_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=existing_portfolio)
    existing_portfolio_returns = existing_portfolio_returns.resample("1W").apply(lambda x: (1 + x).prod() - 1)
    
    optimal_btceth_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=optimal_btc_eth_portfolio)
    optimal_btceth_returns = optimal_btceth_returns.resample("1W").apply(lambda x: (1 + x).prod() - 1)

    df_mcap_sub = df_mcap[df_mcap["timestamp"].dt.date.between(bounds.day0, bounds.day1)].copy()
    df_mcap_sub = df_mcap_sub.dropna().reset_index(drop=True)

    min_size: int = min(df_mcap_sub.shape[0], len(optimal_btceth_returns))

    cov_btceth: float = np.cov(
        df_mcap_sub["market_return"].iloc[:min_size], optimal_btceth_returns.iloc[:min_size]
    )[0, 1]
    cov_existing_portfolio: float = np.cov(
        df_mcap_sub["market_return"].iloc[:min_size], existing_portfolio_returns.iloc[:min_size]
    )[0, 1]
    
    var_market: float = df_mcap_sub["market_return"].std()**2
    
    beta_values.append({
        "days": offset.days,
        "beta_btceth_portfolio": cov_btceth / var_market,
        "beta_existing_portfolio": cov_existing_portfolio / var_market
    })

In [ ]:
pd.DataFrame(beta_values)

In [ ]:
beta_values: List[Dict[str, float]] = []

for offset in tqdm(offsets):
    bounds: Bounds = Bounds.for_days(start_date=date.today() - offset, end_date=date.today())
    
    df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=list(existing_portfolio.keys()))
    df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
    df_returns[Currency.USDT.name] = 0
    # fill missing returns with zeroes
    df_returns = df_returns.fillna(0)

    # Compute returns of BTC/ETH portfolio and existing portfolio
    existing_portfolio_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=existing_portfolio)
    optimal_btceth_returns: pd.Series = compute_return_series_for_portfolio(df_returns=df_returns, portfolio=optimal_btc_eth_portfolio)

    cov_btceth: float = np.cov(optimal_btceth_returns, df_returns["BTC"])[0, 1]
    cov_existing_portfolio: float = np.cov(existing_portfolio_returns, df_returns["BTC"])[0, 1]
    
    var_btc: float = df_returns["BTC"].std()**2
    
    beta_values.append({
        "days": offset.days,
        "beta_btceth_portfolio": cov_btceth / var_btc,
        "beta_existing_portfolio": cov_existing_portfolio / var_btc
    })

<h4>Simulate paths of prices taking into account that they are correlated</h4>

In [ ]:
bounds: Bounds = Bounds.for_days(
    start_date=date.today() - timedelta(days=180), end_date=date.today()
)

df_prices: pd.DataFrame = load_data_from_currencies(bounds=bounds, currencies=list(Currency))

df_returns: pd.DataFrame = compute_log_returns(df_prices=df_prices)
df_returns[Currency.USDT.name] = 0

In [ ]:
mus = df_returns.mean().values
cov = df_returns.cov().values
stddevs = df_returns.std().values

<h4>Simulated return paths for the current portfolio</h4>

In [ ]:
from analysis.backtest.monte_carlo import generate_correlated_gbm, get_portfolio_returns_paths

corr_matrix: np.ndarray = df_returns.corr().values # we need correlation matrix
S0: np.ndarray = np.array(df_prices.iloc[-1, :].tolist() + [1])

St: np.ndarray = generate_correlated_gbm(
    S0=S0, 
    currencies=list(existing_portfolio.keys()), 
    num_steps=365, 
    corr_matrix=corr_matrix,
    stddevs=df_returns.std().values
)

In [ ]:
sim_current_portfolio: np.ndarray = get_portfolio_returns_paths(
    St=St, weights=np.array(list(existing_portfolio.values()))
)

plt.plot(sim_current_portfolio, color="red", alpha=.1)
plt.show()

<h4>Simulated return paths for BTC/ETH portfolio</h4>

In [ ]:
selected_currencies: List[Currency] = [Currency.BTC, Currency.ETH]
cols = [currency.name for currency in selected_currencies]

corr_matrix: np.ndarray = df_returns[cols].corr().values # we need correlation matrix
S0 = df_prices[cols].iloc[-1].values

St: np.ndarray = generate_correlated_gbm(
    S0=S0, 
    currencies=selected_currencies, 
    num_steps=365, 
    corr_matrix=corr_matrix,
    stddevs=df_returns[cols].std().values
)

In [ ]:
sim_btc_eth_portfolio: np.ndarray = get_portfolio_returns_paths(
    St=St, weights=[0.61, 0.39]
)

plt.plot(sim_btc_eth_portfolio, color="yellow", alpha=.1)
plt.show()

In [ ]:
import matplotlib.patches as mpatches

plt.figure(figsize=(16, 10))
plt.plot(sim_current_portfolio, color="red", alpha=.2)
plt.plot(sim_btc_eth_portfolio, color="blue", alpha=.2)

legend_patches = [
    mpatches.Patch(color='red', label="Currently suggested portfolio"),
    mpatches.Patch(color='blue', label="BTC/ETH portfolio"),
]

plt.plot(sim_current_portfolio.mean(axis=1), label="Currently suggested portfolio: Mean return path", color="red", linewidth=2)
plt.plot(sim_btc_eth_portfolio.mean(axis=1), label="BTC/ETH portfolio: Mean return path", color="blue", linewidth=2)

plt.title("Simulated returns of portfolios for 1 year ahead")
plt.legend(handles=legend_patches)
plt.show()

In [ ]:
df_sim: pd.DataFrame = pd.DataFrame({
    "terminal_mean": [sim_current_portfolio[-1, :].mean(), sim_btc_eth_portfolio[-1, :].mean()],
    "terminal_std": [sim_current_portfolio[-1, :].std(), sim_btc_eth_portfolio[-1, :].std()],
    "max_drawdown": [sim_current_portfolio.min(), sim_btc_eth_portfolio.min()],
    "max_return": [sim_current_portfolio.max(), sim_btc_eth_portfolio.max()],
    "sharpe_ratio": [
        (sim_current_portfolio[-1, :].mean()-1) / sim_current_portfolio[-1, :].std(),
        (sim_btc_eth_portfolio[-1, :].mean()-1) / sim_btc_eth_portfolio[-1, :].std()
    ]
})

df_sim.index = [
    "Currently suggested portfolio", "Optimal BTC/ETH portfolio"
]

df_sim